In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from cmapPy.pandasGEXpress.parse import parse
from scipy.stats import spearmanr as scor
from scipy.stats import mannwhitneyu as mwu

Importing the Niche Net matrix

In [2]:
nn_model=pd.read_csv('../data/ligand_target_matrix.csv', sep=',', header=0, index_col=0)

In [3]:
nn_model = nn_model.reindex(sorted(nn_model.columns), axis=1)

In [4]:
nn_model.head()

,A2M,AANAT,ADAM12,ADAM15,ADAM17,ADAM2,ADAM23,ADAM9,ADCYAP1,ADIPOQ,...,WNT7A,WNT7B,WNT8A,WNT8B,WNT9A,WNT9B,XCL1,XCL2,YARS,ZP3
A1BG,0.000386,0.000503,0.000363,0.000407,0.000462,0.000300,0.000279,0.000319,0.000412,0.000638,...,0.000484,0.000463,0.000297,0.000298,0.000298,0.000312,0.000250,0.000237,0.000388,0.000421
A1BG-AS1,0.000196,0.000393,0.000192,0.000343,0.000236,0.000201,0.000186,0.000162,0.000207,0.000346,...,0.000214,0.000221,0.000129,0.000130,0.000124,0.000141,0.000115,0.000113,0.000131,0.000319
A1CF,0.000459,0.000401,0.000523,0.000369,0.000576,0.000355,0.000324,0.000365,0.000469,0.000762,...,0.000652,0.000695,0.000595,0.000616,0.000476,0.000466,0.000329,0.000312,0.000385,0.000466
A2M,0.000770,0.000642,0.000671,0.000646,0.000916,0.000420,0.000427,0.000582,0.000588,0.000833,...,0.000791,0.000875,0.000512,0.000514,0.000509,0.000527,0.000435,0.000396,0.000620,0.000580
A2M-AS1,0.000093,0.000087,0.000087,0.000115,0.000117,0.000137,0.000062,0.000037,0.000171,0.000264,...,0.000036,0.000035,0.000036,0.000006,0.000037,0.000006,0.000082,0.000083,0.000005,0.000138


In [5]:
nn_genes=list(nn_model.index)

Getting the single drugs used in CREEDs 

In [6]:
drug_perturb=pd.read_csv('../data/single_drug_perturbations-v1.0.csv', sep=',', header=0, index_col=0)

In [7]:
drugs=list(drug_perturb['drug_name'].unique())

In [8]:
fout=open ('../data/drugs_perturb.txt', 'w') 
for line in drugs:
    fout.write(line + '\n')
fout.close()

Importing the single drugs used in CREEDs with their receptor targets

In [9]:
drug_targets=pd.read_csv('../data/single_drugs_.csv', sep=';', header=0, index_col=0)

In [10]:
drug_targets #targets from Drugbank

,Target,Target_type
Drugs,,
1.25 dihydroxyvitamin d,VDR,antagonist
1-[3-(trifluoromethyl)phenyl]piperazine,5HT1A,agonist
1-[3-(trifluoromethyl)phenyl]piperazine,5HT1B,agonist
1-[3-(trifluoromethyl)phenyl]piperazine,5HT1D,agonist
1-[3-(trifluoromethyl)phenyl]piperazine,5HT2A,agonist
...,...,...
Vitamin e,NR1I2,NaN
Zinc acetate,BDKRB1,NaN
Zinc acetate,ESR1,NaN


Importing single perturbed genes from CREEDs

In [11]:
gene_perturbed=pd.read_csv('../data/single_gene_perturbations-v1.0.csv', sep=',', header=0, index_col=0)

In [12]:
gene_perturbed.head()

,cell_type,chdir_norm,ctrl_ids,curator,geo_id,hs_gene_symbol,mm_gene_symbol,organism,pert_ids,pert_type,platform,version
id,,,,,,,,,,,,
gene:508,T reg,NaN,GSM998922|GSM998923,dsurujon,GSE40655,FOXO1,Foxo1,mouse,GSM998920|GSM998921,KO,GPL1261,1.0
gene:509,heart,NaN,GSM1080530|GSM1080531|GSM1080532|GSM1080533,dsurujon,GSE44192,PLIN5,Plin5,mouse,GSM1080534|GSM1080535|GSM1080536|GSM1080537,OE,GPL6246,1.0
gene:2349,Inducible stable HEK 293 Flp-In T-REx cells ov...,NaN,GSM997725|GSM997726|GSM997727,gszeto,GSE40601,ERO1L,Ero1l,human,GSM997728|GSM997729|GSM997730,overexpressing a hyperactive mutant,GPL6244,1.0
gene:2348,Inducible stable HEK 293 Flp-In T-REx Ero1alph...,NaN,GSM997719|GSM997720|GSM997721,gszeto,GSE40601,ERO1L,Ero1l,human,GSM997722|GSM997723|GSM997724,OE,GPL6244,1.0
gene:2343,OT-I T cells from spleen,NaN,GSM1290674|GSM1290675|GSM1290676,AliDot,GSE53388,PPP2R2D,Ppp2r2d,mouse,GSM1290677|GSM1290678|GSM1290679,KD,GPL1261,1.0


In [14]:
gene_perturbed.shape

(2176, 12)

Finding the common receptor genes from CREEDs and NicheNet

In [15]:
fil=np.in1d(nn_genes, list(gene_perturbed['hs_gene_symbol']))